In [1]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd

In [7]:
# Read config parser .ini file with your connection information

Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
dotenv.load_dotenv(Path)

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 

connection_parms

# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZJRb9owFIX%2FSuQ9x3YCdGABFQNVRWoHa9J16psXG7Bw7MzXIZRfP4eA1D20D3uLnHPvd%2B49d3x7LHV0kA6UNROUYIoiaQorlNlO0HN%2BFw9RBJ4bwbU1coLeJKDb6Rh4qSs2q%2F3OPMk%2FtQQfhUYGWPtjgmpnmOWggBleSmC%2BYNns8YGlmDIOIJ0POHQpEaACa%2Bd9xQhpmgY3PWzdlqSUUkJHJKhayRf0DlF9zqic9baw%2BlpyDDN9gEgI7beIoAiE9aXwmzLdCj6j%2FO5EwO7zfB2vV1mOotl1urk1UJfSZdIdVCGfnx46AxAcKHNQYanBpcA1xJKDj1PMG8BgbLPRfC8LW1a1D91x%2BCIbKYi2WxV2tlxMULVXQqf%2B5ZXbuv%2BDNjQ%2FLX6dpCpnw%2FuGa9Pr3yQjcfK7l%2FlxdfNYoOjnNeG0TXgJUMulaXP14Ymmg5imMR3kKWW9r2wwxKMefUXRIuSqDPfnyqt5HkLHe1vslDh7U6IiWbbCbSwp6k6DnQlu%2Br8Dj8n7Lpdr%2Bx4CWC7WVqviLbqzruT%2B43wSnJxflIg3ZymTJVd6JoSTACEnrW0zd5L7cNTe1RKRaUf996ynfwE%3D&RelayState=ver%3A1-hint%3A759822674653786-ETMsDgAAAZTX1XgSABRB

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [9]:
session.sql("CREATE SCHEMA IF NOT EXISTS PURCHASING_ANOMALY;").collect()

SnowparkSQLException: (1304): 01ba3256-0002-588d-0002-b30e8780da06: 003001 (42501): SQL access control error:
Insufficient privileges to operate on database 'DS_D_WORKSPACE'